In [2]:
import torch
import torch.nn as nn 
from torch.nn import functional as F
import numpy as np
import inspect

In [10]:
def add(a: int, b: int):
    """
    add two numbers
    """
    # compute the addition
    x = a + b
    return x 

s = inspect.signature(add).parameters['a']
print(s)

a: int


In [3]:
data = open("data/shakespeare_char/input.txt").read()
vocab = sorted(list(set(data)))
print(''.join(vocab))
len(vocab)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


65

In [4]:
import tiktoken
#enc = tiktoken.encoding_for_model('gpt-4o')
enc = tiktoken.get_encoding('gpt2')
print(enc.n_vocab)
codes = enc.encode("Danushka Bollegala")
dec = [enc.decode([cd]) for cd in codes]
print(dec)

50257
['Dan', 'ush', 'ka', ' Bol', 'leg', 'ala']


In [5]:
stoi = {ch:i for (i, ch) in enumerate(vocab)}
itoch = {i:ch for (i, ch) in enumerate(vocab)}
encode = lambda s: [stoi[ch] for ch in s] 
decode = lambda l: ''.join([itoch[i] for i in l])
vocab_size = len(vocab)
print(encode("Danushka Bollegala"))
print(decode(encode("Danushka Bollegala")))

[16, 39, 52, 59, 57, 46, 49, 39, 1, 14, 53, 50, 50, 43, 45, 39, 50, 39]
Danushka Bollegala


In [6]:
n = int(0.9 * len(data))
train_data = torch.tensor(encode(data[:n]))
val_data = torch.tensor(encode(data[n:]))
block_size = 8

In [7]:
torch.manual_seed(1337) # generates the same random sequence
batch_size = 4

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs')
print(xb.shape)
print(xb)

print('targets')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1] # in the b-th batch, we take from the start to length t
        target = yb[b, t] # t-th token in the b-th batch
        print(f"Context = {context.tolist()}, Target = {target}")


inputs
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
Context = [24], Target = 43
Context = [24, 43], Target = 58
Context = [24, 43, 58], Target = 5
Context = [24, 43, 58, 5], Target = 57
Context = [24, 43, 58, 5, 57], Target = 1
Context = [24, 43, 58, 5, 57, 1], Target = 46
Context = [24, 43, 58, 5, 57, 1, 46], Target = 43
Context = [24, 43, 58, 5, 57, 1, 46, 43], Target = 39
Context = [44], Target = 53
Context = [44, 53], Target = 56
Context = [44, 53, 56], Target = 1
Context = [44, 53, 56, 1], Target = 58
Context = [44, 53, 56, 1, 58], Target = 46
Context = [44, 53, 56, 1, 58, 46], Target = 39
Context = [44, 53, 56, 1, 58, 46, 39], Target = 58
Con

In [8]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # nn.Embedding provides one hot encoding. But here we use it as a probability table (current_word --> next_word) for the bigram LM.
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # logis for the next word predictions. (B, T, C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is a (B, T) array of indices in the current context. (B: blocks, T:time positions in each block)
        for _ in range(max_new_tokens): 
            logits, _ = self(idx)
            # We are taking the logits for the last time step
            logits = logits[:,-1,:] # becomes (B, C)
            probs = F.softmax(logits, dim=1) # Converting the values in the C dimension (1st dim) returning (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
        
    
model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)    
print(logits.shape) 
print(loss.item())

idx = torch.zeros((1,1), dtype=torch.long)
print(decode(model.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
5.036386013031006

l-QYjt'CL?jLDuQcLzy'RIo;'KdhpV
vLixa,nswYZwLEPS'ptIZqOZJ$CA$zy-QTkeMk x.gQSFCLg!iW3fO!3DGXAqTsq3pdgq


In [9]:
# create a PyTorch optimiser
batch_size = 32
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
for steps in range(1000000):
    # sample a batch of data
    xb, yb = get_batch('train')
    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

print(decode(model.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

2.395066022872925

Mit,

The, w ck he, ron l d byik hinofiburkighe agl in'd?
PUSCly l llo hbre. on ses fa fou ind n aig


In [35]:
x = torch.rand(size=(2, 3, 4))
print('x', x)
B, T, C = x.shape
y = x.view(-1)
print('y', y)
y[0][0] = 0
print('y', y)
print('x', x)

x tensor([[[0.5733, 0.9822, 0.6841, 0.7220],
         [0.4054, 0.7099, 0.1884, 0.5690],
         [0.4043, 0.9330, 0.8028, 0.7972]],

        [[0.3050, 0.3823, 0.2141, 0.7705],
         [0.8820, 0.1858, 0.4542, 0.1196],
         [0.0182, 0.7626, 0.8931, 0.6682]]])
y tensor([0.5733, 0.9822, 0.6841, 0.7220, 0.4054, 0.7099, 0.1884, 0.5690, 0.4043,
        0.9330, 0.8028, 0.7972, 0.3050, 0.3823, 0.2141, 0.7705, 0.8820, 0.1858,
        0.4542, 0.1196, 0.0182, 0.7626, 0.8931, 0.6682])


IndexError: invalid index of a 0-dim tensor. Use `tensor.item()` in Python or `tensor.item<T>()` in C++ to convert a 0-dim tensor to a number

In [39]:
# concatenating tensors
x = torch.randn(size=(2,2))
y = torch.randn(size=(2,3))
print('x', x)
print('y', y)
z = torch.cat((x,y), dim=0)
print('z', z)

x tensor([[-0.2249,  1.7493],
        [-0.3261, -1.2838]])
y tensor([[-1.7779, -0.7404, -0.8355],
        [-0.5031, -0.8100,  0.3091]])


RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 2 but got size 3 for tensor number 1 in the list.